# Example on the MNIST Digits Dataset

### 1. Import of the modules and the dataset

In [1]:
from neural_network import *
from keras.datasets import mnist

### 2. Load the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### 3. Build the neural network

In [3]:
net = NeuralNetwork(
    Normalization(samples=X_train),
    Reshape(output_shape=(1, 28, 28)),
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
    BatchNorm2d(6),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    BatchNorm2d(16),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Flatten(),
    Linear(in_features=400, out_features=120),
    Tanh(),
    Linear(in_features=120, out_features=84),
    Tanh(),
    Linear(in_features=84, out_features=10),
    SoftmaxCategoricalCrossEntropy()
)
print(net)

NeuralNetwork:
 (0) Normalization(norm=255.0, dtype=float32)
 (1) Reshape(output_shape=(1, 28, 28))
 (2) Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (3) BatchNorm2d(6, eps=1e-05, momentum=0.1)
 (4) Tanh()
 (5) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (6) Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (7) BatchNorm2d(16, eps=1e-05, momentum=0.1)
 (8) Tanh()
 (9) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (10) Flatten()
 (11) Linear(in_features=400, out_features=120, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (12) Tanh()
 (13) Linear(in_features=120, out_features=84, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), ini

### 4. Choose the number of samples to train on

In [4]:
# Select N samples to train on:
N = 3000
X_train, y_train = X_train[:N], y_train[:N]
X_train.shape, y_train.shape

((3000, 28, 28), (3000,))

### 5. Train the neural network

In [5]:
net.fit(X_train, y_train, epochs=5, batch_size=64, shuffle=True)

Training on 3000 samples:
Epoch    1 of 5    	 Average Error = 0.839323 	 Average Accuracy = 76.63%                                   
Epoch    2 of 5    	 Average Error = 0.288938 	 Average Accuracy = 92.20%                                   
Epoch    3 of 5    	 Average Error = 0.189556 	 Average Accuracy = 95.10%                                   
Epoch    4 of 5    	 Average Error = 0.132886 	 Average Accuracy = 96.67%                                   
Epoch    5 of 5    	 Average Error = 0.106323 	 Average Accuracy = 97.50%                                   
Training time : 00 hours, 00 minutes, 22 seconds


### 6. Make predictions on the test samples

In [6]:
# Prediction on the 10,000 test samples:
y_pred = net.predict(X_test, to="labels")
# Shows the first 10 predicted labels vs the true labels:
y_pred[:10], y_test[:10]

(array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9]),
 array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8))

### 7. Evaluate the accuracy score

In [7]:
print(f"Accuracy score on the test set: {accuracy_score(y_test, y_pred):.2%}")

Accuracy score on the test set: 95.52%


### 8. Build a confusion matrix

In [8]:
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

Confusion matrix:
[[ 966    0    2    0    3    0    5    2    2    0]
 [   0 1124    3    2    0    0    3    1    2    0]
 [   9    1  971    5   12    0    9   17    7    1]
 [   3    0   22  947    1    8    0   11    8   10]
 [   0    2    1    0  973    0    3    0    1    2]
 [   4    5    2    3    1  852    4    2   11    8]
 [  12    3    2    0   10    6  923    0    2    0]
 [   1    3   17    1    3    0    0  998    0    5]
 [   7    3   14    6   13    2    6   16  886   21]
 [   4    5    0    3   47   12    1   22    3  912]]


### 9. Displays a classification report

In [9]:
print(classification_report(cm, formatted=True))

┌───────────────────┬───────────┬───────────┬───────────┬───────────┐
│       Class       │ Precision │   Recall  │  F1-Score │  Support  │
├───────────────────┼───────────┼───────────┼───────────┼───────────┤
│         0         │  96.02%   │  98.57%   │  97.28%   │    980    │
│         1         │  98.08%   │  99.03%   │  98.55%   │   1135    │
│         2         │  93.91%   │  94.09%   │  94.00%   │   1032    │
│         3         │  97.93%   │  93.76%   │  95.80%   │   1010    │
│         4         │  91.53%   │  99.08%   │  95.16%   │    982    │
│         5         │  96.82%   │  95.52%   │  96.16%   │    892    │
│         6         │  96.75%   │  96.35%   │  96.55%   │    958    │
│         7         │  93.36%   │  97.08%   │  95.18%   │   1028    │
│         8         │  96.10%   │  90.97%   │  93.46%   │    974    │
│         9         │  95.10%   │  90.39%   │  92.68%   │   1009    │
└───────────────────┴───────────┴───────────┴───────────┴───────────┘

